# SLA1 Camera Characterization

## Testing `ccdproc.combine()` and `ccdproc.subtract()`

On May 8, 2024 (UTC) we took various dark exposures with the [QHY42 Pro](https://www.qhyccd.com/qhy42pro/) camera.

As they are read in, the darks are scaled to undo zero padding (divided by 16) and the effect of gain (multiplied by 1.39).

They are then combined into a master dark.

## Read in and Rescale the Darks


In [1]:
# THIS COMMENT IS THE LONGEST A LINE CAN BE AND STILL RENDER COMPLETELY WHEN PRINTING IN LANDSCAPE MODE.

import os, sys
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection, combine, subtract_dark, flat_correct # Combiner
import astroalign as aa
import matplotlib.pyplot as plt
%matplotlib inline
from math import log10, floor

home_directory = os.path.expanduser('~')

# soft link to directory containing raw images
sessions_directory = os.path.join(home_directory, '2024 SLA Sessions')

uv_project_directory = os.path.join(home_directory, 'Projects', 'uv-transients')
analysis_directory = os.path.join(uv_project_directory, 'analyses', '30s_darks')

# The path to the first dark on SLA1 is D:/Raw/2024-05-08/03_38_48/Dark30s/00001.fits
# The files to be processed need to be mirrored on the local machine
# at ~/2024 SLA Sessions/ using the same subdirectory structure.
capture_date = '2024-05-08'
capture_time = '03_38_48'
object_name = 'Dark30s'

# Amount to scale the image data (typically to undo 0 padding of 12-bit to 16-bit values)

scale_due_to_padding = 2**4  # This is division by 16

scale_due_to_gain = 1.39  # from QHYCCD manual for gain of 5

scale = scale_due_to_gain / scale_due_to_padding

# threshold for flagging hot pixels

threshold = 200

# subdirectory for the 30-second darks (following SharpCap Pro capture directory conventions)
dark_directory = os.path.join(
    sessions_directory,
    capture_date,
    capture_time,
    object_name
)

# exposure duration

dark_exposure = 30.0
dark_exposure_with_ccdproc_units = dark_exposure * u.second

# FITS header confirmation

def confirm_fits_header(image, dimensions, exposure_time, filter):
    header = image.header
    assert header['NAXIS1'] == dimensions[0]
    assert header['NAXIS2'] == dimensions[1]
    assert header['EXPTIME'] == exposure_time
    if filter:
        assert header['FILTER'].rstrip() == filter
        
# Reader with optional parameter to scale (divide) the ADU readings

def scaled_image_reader(file, scale=1):
    img = CCDData.read(file, unit=u.adu)
    scaled_data = img.data * scale
    img.data = scaled_data
    return img

# After all the preliminaries, we read in and combine the dark files

dark_files = ImageFileCollection(dark_directory).files_filtered(include_path='True')

darks = [scaled_image_reader(file, scale=scale) for file in dark_files]

for dark in darks:
    confirm_fits_header(dark, (2048, 2048), dark_exposure, None)


Set MJD-END to 60438.151953 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 60438.151953 from DATE-END'.
Set MJD-END to 60438.152301 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 60438.152301 from DATE-END'.
Set MJD-END to 60438.152648 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 60438.152648 from DATE-END'.
Set MJD-END to 60438.152995 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 60438.152995 from DATE-END'.
Set MJD-END to 60438.153342 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 60438.153342 from DATE-END'.
Set MJD-END to 60438.153689 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 60438.153689 from DATE-END'.
Set MJD-END to 60438.154037 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 60438.154037 from DATE-END'.
Set MJD-END to 60438.154384 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 60438.154384 from DATE-END'.
Set MJD-END to 60438.154731 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 60438.154731 from DATE-END'.
Set MJD-END to 60438.155078 from DATE-END'. [a

## Inspect the Individual Darks

In [2]:
for i in range(len(darks)):
    print('dark from file {}'.format(dark_files[i]))
    print(darks[i].data[0:4, 0:4])

dark from file /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00001.fits
[[5.56000000e+02 5.24360125e+03 2.60625000e-01 0.00000000e+00]
 [4.98749375e+02 4.98749375e+02 1.54055437e+03 1.63715937e+03]
 [1.50919250e+03 1.54793875e+03 1.58685875e+03 1.67651375e+03]
 [1.53499437e+03 1.47426875e+03 1.65974687e+03 2.10446000e+03]]
dark from file /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00002.fits
[[5.56000000e+02 5.26584125e+03 3.47500000e-01 0.00000000e+00]
 [4.98749375e+02 4.98749375e+02 1.53317000e+03 1.61292125e+03]
 [1.45046500e+03 1.53317000e+03 1.55541000e+03 1.67277812e+03]
 [1.35229625e+03 1.51284125e+03 1.61292125e+03 2.15154625e+03]]
dark from file /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00003.fits
[[5.56000000e+02 5.28808125e+03 4.34375000e-01 0.00000000e+00]
 [4.98749375e+02 4.98749375e+02 1.54611437e+03 1.55167437e+03]
 [1.48156625e+03 1.56461875e+03 1.52387437e+03 1.72933375e+03]
 [1.40311812e+03 1.51657687e+03 1.63715937e+03 2.0

## Combine the Darks into a Master Dark

In [3]:
combination_method = 'median'  # alternatively, the method can be 'average'

master_dark = combine(darks, method=combination_method)


## Inspect the Data of the Master Dark

In [4]:
# np.set_printoptions(threshold=sys.maxsize) # Uncommenting this line will cause serious I/O strain

master_dark.data[0:4, 0:4]


array([[5.56000000e+02, 5.34368125e+03, 3.04062500e-01, 0.00000000e+00],
       [4.98749375e+02, 4.98749375e+02, 1.52947781e+03, 1.58694562e+03],
       [1.49537937e+03, 1.55445437e+03, 1.55723438e+03, 1.68033625e+03],
       [1.46601562e+03, 1.47518094e+03, 1.61670031e+03, 2.06927562e+03]])

### Are the Master Dark Pixel Values as Expected?

Yes, checked by hand with the data below.

In [5]:
print('for master dark')
print('pixel[0, 1] (x = 1, y = 0) = {}'.format(master_dark.data[0, 1]))

for i in range(len(darks)):
    print('for dark {}'.format(dark_files[i]))
    print('pixel[0, 1] (x = 1, y = 0) = {}'.format(darks[i].data[0, 1]))

for master dark
pixel[0, 1] (x = 1, y = 0) = 5343.68125
for dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00001.fits
pixel[0, 1] (x = 1, y = 0) = 5243.60125
for dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00002.fits
pixel[0, 1] (x = 1, y = 0) = 5265.8412499999995
for dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00003.fits
pixel[0, 1] (x = 1, y = 0) = 5288.081249999999
for dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00004.fits
pixel[0, 1] (x = 1, y = 0) = 5310.32125
for dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00005.fits
pixel[0, 1] (x = 1, y = 0) = 5332.56125
for dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00006.fits
pixel[0, 1] (x = 1, y = 0) = 5354.8012499999995
for dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00007.fits
pixel[0, 1] (x = 1, y = 0) = 5377.041249999999
for dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00008.fits
pi

## Subtract the Darks

In [6]:
subtracted_darks = [
    subtract_dark(
        dark,
        master_dark,
        data_exposure=dark_exposure_with_ccdproc_units,
        dark_exposure=dark_exposure_with_ccdproc_units,
        scale=False
    )
    for dark in darks
]


### Are the Subtracted Pixel Values as Expected

In [7]:
print('for master dark')
print('pixel[0, 1] (x = 1, y = 0) = {}'.format(master_dark.data[0, 1]))

for i in range(len(darks)):
    print('for subtracted dark {}'.format(dark_files[i]))
    print('pixel[0, 1] (x = 1, y = 0) = {}'.format(subtracted_darks[i].data[0, 1]))


for master dark
pixel[0, 1] (x = 1, y = 0) = 5343.68125
for subtracted dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00001.fits
pixel[0, 1] (x = 1, y = 0) = -100.07999999999993
for subtracted dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00002.fits
pixel[0, 1] (x = 1, y = 0) = -77.84000000000015
for subtracted dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00003.fits
pixel[0, 1] (x = 1, y = 0) = -55.600000000000364
for subtracted dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00004.fits
pixel[0, 1] (x = 1, y = 0) = -33.35999999999967
for subtracted dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00005.fits
pixel[0, 1] (x = 1, y = 0) = -11.11999999999989
for subtracted dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00006.fits
pixel[0, 1] (x = 1, y = 0) = 11.11999999999989
for subtracted dark /Users/brian/2024 SLA Sessions/2024-05-08/03_38_48/Dark30s/00007.fits
pixel[0, 1] (x = 1, y = 0